In [10]:
import pickle
import numpy as np
import moteus
import pandas as pd
import matplotlib.pyplot as plt
import plotly

try:
  import scipy.signal as signal
  import scipy.integrate as integrate
except ImportError:
  pass

In [11]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

In [28]:
def cumtrapz(y, x):
  dt = np.diff(x)
  dt = np.append(dt, dt[-1])
  y_normalized = y * dt
  return np.cumsum(y_normalized)

def process_data(data, start_idx=None, end_idx=None):
    data_subset = data[start_idx:end_idx]
    torques = np.array([datum[1].values[moteus.Register.TORQUE] for datum in data_subset])
    ts = np.array([datum[0] for datum in data_subset])
    positions = np.array([datum[1].values[moteus.Register.POSITION] for datum in data_subset])
    velocities = np.array([datum[1].values[moteus.Register.VELOCITY] for datum in data_subset])
    accel = np.concatenate(([0], np.diff(velocities))) / np.mean(np.diff(ts))
    accel_filt = accel
    try:
      sos = signal.butter(2, 0.25, output='sos')
      accel_filt = signal.sosfiltfilt(sos, accel)
    except Exception:
      pass
    print("dt: ", np.mean(np.diff(ts)))
    powers = torques * velocities * 2 * np.pi # Convert to W
    energy = np.zeros_like(powers)
    try:
      energy = integrate.cumulative_trapezoid(y=powers, x=ts) # J
    except Exception:
      pass
    return {"ts":ts, 
            "positions":positions, 
            "velocities":velocities, 
            "accels":accel_filt, 
            "torques":torques, 
            "powers":powers,
            "energy":energy}

In [29]:
y = np.array([1,1,1])
x = np.array([1,1.1,3])
cumtrapz(y, x)

array([0.1, 2. , 3.9])

In [13]:
def plot_data(data_dict):
    fig = sp.make_subplots(rows=len(data_dict), cols=1, shared_xaxes=True, vertical_spacing = 0.01)
    fig_combined = go.Figure()
    x = data_dict["ts"]
    for i, (key, data) in enumerate(data_dict.items()):
        if key == "ts":
            continue
        fig.add_trace(go.Scatter(name=key, x=x, y=data), row=i, col=1)
        fig_combined.add_trace(go.Scatter(name=key, x=x, y=data/np.max(data)))
    fig.update_layout(height=100*len(data_dict), width=800)
    fig.show()
    fig_combined.update_layout(height=100*len(data_dict), width=800)
    fig_combined.show()

def compare_data(data1, data2):
    fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing = 0.01)
    fig.add_trace(go.Scatter(name='torque_1', x=data1["ts"], y=data1["torques"]), row=1, col=1)
    fig.add_trace(go.Scatter(name='torque_2', x=data2["ts"], y=data2["torques"]), row=1, col=1)
    fig.add_trace(go.Scatter(name='vel_1', x=data1["ts"], y=data1["velocities"]), row=2, col=1)
    fig.add_trace(go.Scatter(name='vel_2', x=data2["ts"], y=data2["velocities"]), row=2, col=1)
    fig.add_trace(go.Scatter(name='pos_1', x=data1["ts"], y=data1["positions"]), row=3, col=1)
    fig.add_trace(go.Scatter(name='pos_2', x=data2["ts"], y=data2["positions"]), row=3, col=1)
    fig.update_layout(height=600, width=800)
    fig.show()

In [14]:
from IPython.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

# Repeated jump

In [19]:
data_noload = pickle.load(open("logs/log_4_22_jump.pickle","rb"))
data_noload = process_data(data_noload)#, 200, 300)
plot_data(data_noload)

dt:  0.012982245454090876


NameError: name 'integrate' is not defined

In [148]:
compare_data(data_noload, data_jump1)